In [0]:
df1 = spark.sql ("select count(*) from UserSCV")
df1.show()

+--------+
count(1)|
+--------+
 600|
+--------+

In [0]:
# import libraries
from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType, DoubleType, StructType, StructField
import requests
import json
import phonenumbers 
import re
import datetime
import pandas as pd
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import  col
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit




In [0]:
# SQL server connection Details ----------
server = 'sqldbplatingnum.database.windows.net'
database = 'cozydb'
username = 'platingnum'
password = 'D0wnl0ad!'
driver= '{ODBC Driver 13 for SQL Server}'
connectionProperties = {
  "user" : username,
  "password" : password
}

connection_string = 'Driver=' + driver + ';Server=' + server + ';Database=' + database + ';Trusted_Connection=Yes;UID=' + username + ';PWD=' + password + ';'
jdbc_url = "jdbc:sqlserver://"+ server +";database="+ database +";user="+ username +";password="+ password

In [0]:
# schema for user
user_schema = StructType([
            StructField("id", IntegerType(), False),
            StructField("Userid", IntegerType(), True),
            StructField("SkinID", StringType(), True),
            StructField("username", StringType(), True),
            StructField("first_name", StringType(), True),
            StructField("last_name", StringType(), True),
            StructField("email", StringType(), True),
            StructField("gender", StringType(), True), 
            StructField("ip_address", StringType(), True), 
            StructField("RegDate", StringType(), True), 
            StructField("RegIP", StringType(), True), 
            StructField("LastIP", StringType(), True), 
            StructField("DOB", StringType(), True), 
            StructField("Postcode", StringType(), True), 
            StructField("MobilePhone", StringType(), True), 
            StructField("Landline", StringType(), True), 
            StructField("Address1", StringType(), True),
            StructField("City", StringType(), True),
            StructField("County", StringType(), True),
            StructField("Country", StringType(), True),
            StructField("SelfExcludedUntil", StringType(), True),
            StructField("Status", StringType(), True)])
            

In [0]:
def fixUserRow(c):
    # get the Mobile field
    number = c.MobilePhone

    # initialize variables 
    is_valid_number = "N"
    clean_number = None
    number_type = None
    valid_mail = None

    p = None

    if number is not None:
        # Clean the Mobile Number first
        try:
            p = phonenumbers.parse(number, c.Country)

            if phonenumbers.is_valid_number(p):
                is_valid_number = "Y"
            elif phonenumbers.truncate_too_long_number(p):
                is_valid_number = "Y"
            else:
                is_valid_number = "N"

            clean_number = "%s%s" % (p.country_code, p.national_number)
            
        except:
            p = None

    # clean up PhoneNumber
    phone_no = c.Landline
    if phone_no is not None:
      phone_no = phone_no.replace('-', '')
      if (len(phone_no) != 10):
        phone_no = None
    
    # validate Email 
    if re.match(r"^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$", c.email):
      valid_mail = c.email
    
    return Row( 
		id = c.id, 
        Userid = c.Userid, 			
        SkinID = c.SkinID,
        username = c.username,
        first_name = c.first_name, 		
        last_name = c.last_name,	
        email = valid_mail,			
        gender = c.gender,			
        ip_address = c.ip_address,
        RegDate = c.RegDate,
        RegIP = c.RegIP,
		LastIP = c.LastIP,			
		DOB = c.DOB,			
		Postcode = c.Postcode,		
		MobilePhone = clean_number, 	
		Landline = phone_no, 		
		Address1 = c.Address1,		
        City = c.City, 			
		County = c.County,			
		Country = c.Country, 		
        SelfExcludedUntil = c.SelfExcludedUntil,
		Status = c.Status			
    )


In [0]:
csv_url = "https://my.api.mockaroo.com/users_load.json?key=6af9c3e0"
# Create dataframe by reading values from csv
df_user = spark.createDataFrame(pd.read_csv(csv_url))

In [0]:
# cleanse the data
df_user_updated = df_user.rdd.map(lambda c: fixUserRow(c))
# change the column type now
df = sqlContext.createDataFrame(df_user_updated, user_schema)
df = df.select("id","Userid", "SkinID", \
               #df.SkinID.cast(IntegerType()).alias('SkinID')",
               "username","first_name", 
               "last_name","email","gender","ip_address","RegDate","RegIP","LastIP",\
               #from_unixtime(unix_timestamp("DOB", "dd-MMM-yyyy")).alias("DOB"), \
               #df.DOB.cast(DateType()).alias('DOB'),
               "DOB",
               "Postcode","MobilePhone","Landline","Address1","City","County","Country", \
               #from_unixtime(unix_timestamp("SelfExcludedUntil", "dd-MMM-yyyy")).alias("SelfExcludedUntil"),
               "SelfExcludedUntil", "Status")
  

In [0]:
df.show()

+---+------+---------+-------------+----------+------------+--------------------+------+---------------+----------+--------------+---------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+
 id|Userid| SkinID| username|first_name| last_name| email|gender| ip_address| RegDate| RegIP| LastIP| DOB|Postcode|MobilePhone| Landline| Address1| City|County|Country|SelfExcludedUntil|Status|
+---+------+---------+-------------+----------+------------+--------------------+------+---------------+----------+--------------+---------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+
 1| 1| Orange| aferrulli0| Aaron| Ferrulli|aferrulli0@netvib...| Male| 229.44.43.242|09/25/2017| 152.152.236.6| 151.98.125.162|12/14/1995| 10275|12128688819|5186061616| 27672 Porter Avenue|New York City| NY| US| 2/23/2018| false|
 2| 2| Orange| fjeafferson1| Freida| Jeafferson|fjeafferson1@crai...|Female|175.153.148.129|12/10/2017|205.209.111.99| 71.106.193.10|02/19/1997| 13217|13158124228|2129627120|307 Pankratz Parkway| Syracuse| NY| US| 12/2/2017| true|
 3| 3| Khaki| dperrington2| Drucy| Perrington|dperrington2@payp...|Female| 209.3.107.190|01/04/2018|25.193.145.243| 16.75.149.27|09/18/1984| 12242|15183463493|6466264991|1763 New Castle Road| Albany| NY| US| 2/19/2018| true|
 4| 4| Orange| bpletts3| Benton| Pletts|bpletts3@rakuten....| Male|120.116.139.155|07/14/2017|167.13.249.109|228.237.223.135|01/13/1979| 10160|12129919370|3153937010| 6 Vera Lane|New York City| NY| US| 7/14/2017| false|
 5| 5| Purple| kmattimoe4| Krispin| Mattimoe|kmattimoe4@hc360.com| Male|199.240.235.203|06/16/2017| 59.30.48.254| 176.224.72.186|03/02/1975| 13224|13157920134|9148648884| 8 Welch Junction| Syracuse| NY| US| 5/4/2017| false|
 6| 6| Purple|kleithharvey5| Kippy|Leith-Harvey|kleithharvey5@fee...| Male| 237.27.25.15|09/08/2017|174.201.124.23| 79.238.86.85|03/30/1987| 14619|15852403488|2126007202|23588 Pine View J...| Rochester| NY| US| 2/1/2018| true|
 7| 7| Yellow| jpell6| Julina| Pell| jpell6@bbb.org|Female| 167.35.215.34|02/27/2017|112.176.96.161| 8.83.211.184|05/17/1968| 14683|15852593378|2122669397|96 Summer Ridge D...| Rochester| NY| US| 10/4/2017| false|
 8| 8| Yellow| bgroveham7| Bevan| Groveham|bgroveham7@shutte...| Male| 142.124.29.176|11/18/2017|162.116.64.126| 172.184.66.93|02/18/1981| 13224|13153532871|3472101799|4091 Declaration ...| Syracuse| NY| US| 1/25/2018| true|
 9| 9| Teal| jmcgiffie8| Jacquelin| McGiffie|jmcgiffie8@washin...|Female| 189.119.104.36|12/22/2017|151.160.70.145| 180.67.78.146|08/10/1986| 14276|17162077996|3159991983| 258 Duke Lane| Buffalo| NY| US| 5/16/2017| false|
 10| 10|Turquoise| sgrundey9| Sydelle| Grundey| sgrundey9@home.pl|Female| 249.27.153.70|02/10/2018|166.61.220.252| 243.187.110.79|05/21/1978| 10079|12122393847|6462144214| 935 Riverside Alley|New York City| NY| US| 3/6/2018| false|
 11| 11| Maroon| eklaassensa| Ediva| Klaassens|eklaassensa@yelp.com|Female| 153.134.14.123|09/10/2017| 224.72.89.219| 104.38.120.242|12/21/1971| 14215|17165416866|7186518782| 3 Grasskamp Place| Buffalo| NY| US| 5/5/2017| false|
 12| 12|Goldenrod| kgrouseb| Koo| Grouse| kgrouseb@wufoo.com|Female|109.232.226.137|12/06/2017| 61.193.87.50| 148.90.206.228|12/12/1971| 11407|15161574946|6467225800| 15335 Ruskin Lane| Jamaica| NY| US| 11/30/2017| false|
 13| 13| Violet| tfaroc| Tymothy| Faro| tfaroc@unesco.org| Male| 174.234.59.145|08/13/2017| 79.109.32.227| 51.197.50.12|10/18/1980| 10131|12126016594|7169858995|89 Mayfield Junction|New York City| NY| US| 11/17/2017| true|
 14| 14| Khaki| baddicoted| Babs| Addicote|baddicoted@cocolo...|Female| 242.145.93.92|05/21/2017| 79.36.171.221| 5.144.54.34|10/27/1983| 10045|12125285001|5858526271|94658 Farragut Ju...|New York City| NY| US| 9/24/2017| false|
 15| 15| Maroon| sdeleone| Sol| de Leon| sdeleone@sun.com| Male| 112.81.83.65|03/18/2017| 173.81.238.20| 156.119.83.21|11/11/

In [0]:
# write df to db
# df.write.jdbc(jdbc_url, 'users_load', mode='overwrite', properties=connectionProperties)
df.write.saveAsTable("users_load")


In [0]:
# create output table



In [0]:
dateTimeStr = datetime.datetime.today().strftime("%m-%d-%Y %H:%M:%S")


userSCV =  df.withColumn("ID", F.monotonically_increasing_id()) \
.withColumn("RelatedID", lit(-1).cast(IntegerType())) 
userSCV = userSCV.withColumn("EntityId", col("ID")) 

userSCV = userSCV.withColumn("OriginalEmail", col("email")) 
userSCV = userSCV.withColumn("OriginalFirstname", col("first_name")) 
userSCV = userSCV.withColumn("OriginalLastname", col("last_name")) 
userSCV = userSCV.withColumn("OriginalRegDate", col("RegDate"))
userSCV = userSCV.withColumn("OriginalDOB", col("DOB"))
userSCV = userSCV.withColumn("OriginalPostcode", col("Postcode"))             
userSCV = userSCV.withColumn("OriginalMobilePhone", col("MobilePhone"))
userSCV = userSCV.withColumn("OriginalAddress1", col("Address1"))            
#userSCV = userSCV.withColumn("OriginalAddress2", col("Address2"))            
userSCV = userSCV.withColumn("OriginalCity", col("City"))
userSCV = userSCV.withColumn("Firstname_Lastname_RegIP", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('RegIP') ))          
userSCV = userSCV.withColumn("Firstname_Lastname_LastIP", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('LastIP') ))
userSCV = userSCV.withColumn("Firstname_Lastname_Username", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('Username') ))
userSCV = userSCV.withColumn("Firstname_DOB_City", F.concat(col('first_name'),lit('_'), col('DOB'), lit('_'),col('City') ))
userSCV = userSCV.withColumn("Firstname_Postcode", F.concat(col('first_name'),lit('_'), col('Postcode')  )) 
userSCV = userSCV.withColumn("Firstname_Mobilephone", F.concat(col('first_name'),lit('_'), col('MobilePhone')  ))          
userSCV = userSCV.withColumn("DOB_Postcode", F.concat(col('DOB'),lit('_'), col('Postcode')  )) 
userSCV = userSCV.withColumn("Address1_Postcode", F.concat(col('Address1'),lit('_'), col('Postcode')  ))              
userSCV = userSCV.withColumn("Firstname_Lastname_Address1_City", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('Address1'), lit('_'), col('City') ))
userSCV = userSCV.withColumn("Load_date", lit(dateTimeStr))
userSCV = userSCV.withColumn("LastModifiedDate", lit(dateTimeStr))
userSCV = userSCV.withColumn("CompareStatus", lit(0))
userSCV = userSCV.withColumn("CompareStatus", lit(None).cast(StringType()))
#userSCV = col("ID").alias("ENTITYID")



                    #df.withColumn("RelatedID", lit(-1).cast(IntegerType())), \
                    #df.username, df.email, col("email").alias("originalemail"), \
                    #df.first_name, col("first_name").alias("originalemail"))
                    


In [0]:
userSCV.show(5)

+---+------+------+------------+----------+----------+--------------------+------+---------------+----------+--------------+---------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+---------+--------+--------------------+-----------------+----------------+---------------+-----------+----------------+-------------------+--------------------+-------------+------------------------+-------------------------+---------------------------+--------------------+------------------+---------------------+----------------+--------------------+--------------------------------+-------------------+-------------------+-------------+
 ID|Userid|SkinID| username|first_name| last_name| email|gender| ip_address| RegDate| RegIP| LastIP| DOB|Postcode|MobilePhone| Landline| Address1| City|County|Country|SelfExcludedUntil|Status|RelatedID|EntityId| OriginalEmail|OriginalFirstname|OriginalLastname|OriginalRegDate|OriginalDOB|OriginalPostcode|OriginalMobilePhone| OriginalAddress1| OriginalCity|Firstname_Lastname_RegIP|Firstname_Lastname_LastIP|Firstname_Lastname_Username| Firstname_DOB_City|Firstname_Postcode|Firstname_Mobilephone| DOB_Postcode| Address1_Postcode|Firstname_Lastname_Address1_City| Load_date| LastModifiedDate|CompareStatus|
+---+------+------+------------+----------+----------+--------------------+------+---------------+----------+--------------+---------------+----------+--------+-----------+----------+--------------------+-------------+------+-------+-----------------+------+---------+--------+--------------------+-----------------+----------------+---------------+-----------+----------------+-------------------+--------------------+-------------+------------------------+-------------------------+---------------------------+--------------------+------------------+---------------------+----------------+--------------------+--------------------------------+-------------------+-------------------+-------------+
 0| 1|Orange| aferrulli0| Aaron| Ferrulli|aferrulli0@netvib...| Male| 229.44.43.242|09/25/2017| 152.152.236.6| 151.98.125.162|12/14/1995| 10275|12128688819|5186061616| 27672 Porter Avenue|New York City| NY| US| 2/23/2018| false| -1| 0|aferrulli0@netvib...| Aaron| Ferrulli| 09/25/2017| 12/14/1995| 10275| 12128688819| 27672 Porter Avenue|New York City| Aaron_Ferrulli_15...| Aaron_Ferrulli_15...| Aaron_Ferrulli_af...|Aaron_12/14/1995_...| Aaron_10275| Aaron_12128688819|12/14/1995_10275|27672 Porter Aven...| Aaron_Ferrulli_27...|05-08-2018 05:45:40|05-08-2018 05:45:40| null|
 1| 2|Orange|fjeafferson1| Freida|Jeafferson|fjeafferson1@crai...|Female|175.153.148.129|12/10/2017|205.209.111.99| 71.106.193.10|02/19/1997| 13217|13158124228|2129627120|307 Pankratz Parkway| Syracuse| NY| US| 12/2/2017| true| -1| 1|fjeafferson1@crai...| Freida| Jeafferson| 12/10/2017| 02/19/1997| 13217| 13158124228|307 Pankratz Parkway| Syracuse| Freida_Jeafferson...| Freida_Jeafferson...| Freida_Jeafferson...|Freida_02/19/1997...| Freida_13217| Freida_13158124228|02/19/1997_13217|307 Pankratz Park...| Freida_Jeafferson...|05-08-2018 05:45:40|05-08-2018 05:45:40| null|
 2| 3| Khaki|dperrington2| Drucy|Perrington|dperrington2@payp...|Female| 209.3.107.190|01/04/2018|25.193.145.243| 16.75.149.27|09/18/1984| 12242|15183463493|6466264991|1763 New Castle Road| Albany| NY| US| 2/19/2018| true| -1| 2|dperrington2@payp...| Drucy| Perrington| 01/04/2018| 09/18/1984| 12242| 15183463493|1763 New Castle Road| Albany| Drucy_Perrington_...| Drucy_Perrington_...| Drucy_Perrington_...|Drucy_09/18/1984_...| Drucy_12242| Drucy_15183463493|09/18/1984_12242|1763 New Castle R...| Drucy_Perrington_...|05-08-2018 05:45:40|05-08-2018 05:45:40| null|
 3| 4|Orange| bpletts3| Benton| Pletts|bpletts3@rakuten....| Male|120.116.139.155|07/14/2017|167.13.249.109|228.237.223.135|01/13/1979| 10160|12129919370|3153937010| 6 Vera Lane|New York City| NY| US| 7/14/2017| false| -1| 3|bpletts3@rakuten....| Benton| Pletts| 07/14

In [0]:
# Create output Table
userSCV.write.saveAsTable("UserSCV")


In [0]:
# now read the next load from the JSON url
# and compare it with userSCV table for the matching criteria
csv_url = "https://my.api.mockaroo.com/users_load.json?key=6af9c3e0"
# Create dataframe by reading values from csv
df_user = spark.createDataFrame(pd.read_csv(csv_url))

In [0]:
# cleanse the data
df_user_updated1 = df_user.rdd.map(lambda c: fixUserRow(c))
# change the column type now
df_new = sqlContext.createDataFrame(df_user_updated1, user_schema)
df_new = df_new.select("id","Userid", "SkinID", \
               #df.SkinID.cast(IntegerType()).alias('SkinID')",
               "username","first_name", 
               "last_name","email","gender","ip_address","RegDate","RegIP","LastIP",\
               #from_unixtime(unix_timestamp("DOB", "dd-MMM-yyyy")).alias("DOB"), \
               #df.DOB.cast(DateType()).alias('DOB'),
               "DOB",
               "Postcode","MobilePhone","Landline","Address1","City","County","Country", \
               #from_unixtime(unix_timestamp("SelfExcludedUntil", "dd-MMM-yyyy")).alias("SelfExcludedUntil"),
               "SelfExcludedUntil", "Status")

In [0]:
# now we have base userSCV and next load data to be compared
# check for FirstName, LastName, DOB, Email and Postcode
df_criteria1 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name) & (userSCV.last_name == df_new.last_name) \
                 & (userSCV.DOB == df_new.DOB) & (userSCV.email == df_new.email) \
                 & (userSCV.Postcode == df_new.Postcode))


In [0]:
df_criteria1.count()


Out[ 32 ]: 0

In [0]:
df_new.count()


Out[ 33 ]: 100

In [0]:
userSCV.count()

Out[ 34 ]: 100

In [0]:
# check for LastName, DOB, Email and Postcode
df_criteria2 = userSCV.join(df_new,  (userSCV.last_name == df_new.last_name) \
                 & (userSCV.DOB == df_new.DOB) & (userSCV.email == df_new.email) \
                 & (userSCV.Postcode == df_new.Postcode))
df_criteria2.count()

Out[ 35 ]: 0

In [0]:
# check for FirstName, DOB, Email and Postcode
df_criteria3 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name)  \
                 & (userSCV.DOB == df_new.DOB) & (userSCV.email == df_new.email) \
                 & (userSCV.Postcode == df_new.Postcode))


In [0]:
df_criteria3.count()

Out[ 37 ]: 0

In [0]:
# check for FirstName, lastName, Email and Postcode
df_criteria4 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name)  & (userSCV.last_name == df_new.last_name) \
                 &  (userSCV.email == df_new.email) \
                 & (userSCV.Postcode == df_new.Postcode))
df_criteria4.count()

Out[ 38 ]: 0

In [0]:
# check for FirstName, LastName, DOB and Postcode
df_criteria5 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name) & (userSCV.last_name == df_new.last_name) \
                 & (userSCV.DOB == df_new.DOB)  \
                 & (userSCV.Postcode == df_new.Postcode))
df_criteria5.count()

Out[ 39 ]: 0

In [0]:
# check for FirstName, LastName, DOB, Email and Postcode
df_criteria6 = userSCV.join(df_new, (userSCV.first_name == df_new.first_name) & (userSCV.last_name == df_new.last_name) \
                 & (userSCV.DOB == df_new.DOB) & (userSCV.email == df_new.email) )
df_criteria6.count()


Out[ 40 ]: 0

In [0]:
# check for the minimal condition
# whether firstName and IP equals

df_criteria_min = userSCV.join(df_new, (userSCV.first_name == df_new.first_name) & (userSCV.ip_address == df_new.ip_address) )
df_criteria_min.count()

Out[ 43 ]: 0

In [0]:
# check for email alone
df_criteria_email = userSCV.join(df_new, (userSCV.email == df_new.email))
df_criteria_email.count()

Out[ 44 ]: 0